In [4]:
from functools import cached_property

# **Principaux décorateurs**

## **@classmethod**

**Fonctionnement**<br>
La méthode n'appelle pas l'objet instancié, def fonction(self), mais la classe directement, def fonction(cls).<br>
Utile pour modifier des attributs de classe dynamiquement.<br>

**Utilité**
- **Gestion des attributs de classe** : Comme dans l’exemple ci-dessus, une méthode de classe peut être utilisée pour gérer des attributs partagés par toutes les instances de la classe.
- **Constructeurs alternatifs** : Les méthodes de classe peuvent servir de constructeurs alternatifs. Par exemple, vous pouvez avoir une méthode de classe qui initialise une instance de différentes manières.
- **Validateur avec Pydantic**
- **Polymorphisme** : Les méthodes de classe permettent d’utiliser le polymorphisme, car elles peuvent être redéfinies dans les sous-classes et appelées de manière appropriée.

In [ ]:
# DEMO : Gestion des attributs de classe
class CompteurInstances:
    nombre_instances = 0

    def __init__(self):
        CompteurInstances.incrementer_compteur() # Appele la méthode de classe à chaque instanciation d'objets

    @classmethod
    def incrementer_compteur(cls):
        cls.nombre_instances += 1

print("0. Attribut nombre_instances après définition de la classe sans instancier d'objets", CompteurInstances.nombre_instances)  # Affiche 3

# Création de plusieurs instances
a = CompteurInstances()
print("1. Attribut de l'objet a instancié", a.nombre_instances)
b = CompteurInstances()
print("2. Attribut de l'objet b instancié", b.nombre_instances)
c = CompteurInstances()
print("3. Attribut de l'objet c instancié", c.nombre_instances)

print("3. Attribut directement appelé sur la classe", CompteurInstances.nombre_instances)  # Affiche 3

for idx, (instance, name) in enumerate(zip((a, b, c), ("a", "b", "c"))):
    print("3. Vérification sur", name,  instance.nombre_instances)

0. Attribut nombre_instances après définition de la classe sans instancier d'objets 0
1. Attribut de l'objet a instancié 1
2. Attribut de l'objet b instancié 2
3. Attribut de l'objet c instancié 3
3. Attribut directement appelé sur la classe 3
3. Vérification sur a 3
3. Vérification sur b 3
3. Vérification sur c 3


In [ ]:
# DEMO : Constructeurs alternatifs (pour créer une instance outrepassant __init__)
class Personne:
    def __init__(self, prenom, nom):
        self.prenom = prenom
        self.nom = nom

    @classmethod
    def depuis_nom_complet(cls, nom_complet):
        prenom, nom = nom_complet.split()
        return cls(prenom, nom)

    @classmethod
    def depuis_initiales(cls, initiales):
        prenom, nom = initiales.split('.')
        return cls(prenom, nom)

# Utilisation des constructeurs alternatifs
personne1 = Personne.depuis_nom_complet("Jean Dupont")
personne2 = Personne.depuis_initiales("J.Dupont")

print(f"Personne 1: {personne1.prenom} {personne1.nom}")  # Affiche: Personne 1: Jean Dupont
print(f"Personne 2: {personne2.prenom} {personne2.nom}")  # Affiche: Personne 2: J Dupont


In [20]:
# DEMO : Validateur avec Pydantic
from pydantic import BaseModel, Field, ValidationError

class Utilisateur(BaseModel):
    nom: str
    age: int

    @classmethod
    def depuis_chaine(cls, chaine: str):
        try:
            nom, age = chaine.split(',')
            return cls(nom=nom.strip(), age=int(age.strip()))
        except ValueError as e:
            raise ValidationError(f"Erreur de validation: {e}")

# Utilisation
try:
    utilisateur = Utilisateur.depuis_chaine("Alice, 30")
    print(utilisateur)
except ValidationError as e:
    print(e)

try:
    utilisateur_invalide = Utilisateur.depuis_chaine("Bob, trente")
except ValidationError as e:
    print(e)  # Affiche une erreur de validation



nom='Alice' age=30


TypeError: ValidationError.__new__() missing 1 required positional argument: 'line_errors'

In [ ]:
# DEMO : POLYMORPHISME est utilisable également avec les @classmethod
class Animal:
    def __init__(self, nom):
        self.nom = nom

    @classmethod
    def parler(cls):
        raise NotImplementedError("Cette méthode doit être redéfinie dans les sous-classes")

class Chien(Animal):
    @classmethod
    def parler(cls):
        return "Woof!"

class Chat(Animal):
    @classmethod
    def parler(cls):
        return "Meow!"

# Utilisation du polymorphisme
animaux = [Chien("Rex"), Chat("Whiskers")]

for animal in animaux:
    print(f"{animal.nom} dit {animal.parler()}")


Rex dit Woof!
Whiskers dit Meow!


## **@staticmethod**

**Fonctionnement**<br>
La staticmethod n'appelle ni l'objet instancié (self) ni la classe (cls).<br>
Elle peut lier des fonctions dont le concept est lié à la classe.<br>
Une classe peut ainsi regrouper des fonctions utilitaires pour mieux organiser son code ou aussi être utilisé pour des méthodes de validation utiles pour la classe.

**Utilité**
- **Fonctions utilitaires de classe**
- **Regroupement de fonctions utilitaires sous un nom de classe**
- **Méthodes de validation** : Avec ou sans Pydantic. Moins flexible qu'avec classmethod qui fait appel à cls.
- **Méthodes de fabrique** : Idem que constructeur alternatif avec @classmethod mais utilise le constructeur de base et ajoute un wrapper dessus.

In [9]:
# DEMO - Fonction utilitaire de classe
class MathUtils:
    @staticmethod
    def addition(a, b):
        return a + b

# Utilisation
resultat = MathUtils.addition(5, 3)
print(resultat)  # Affiche 8


8


In [10]:
# DEMO - Regroupement de fonctions utilitaires sous un nom de classe
class Convertisseur:
    @staticmethod
    def celsius_vers_fahrenheit(celsius):
        return celsius * 9/5 + 32

# Utilisation
print(Convertisseur.celsius_vers_fahrenheit(25))  # Affiche 77.0


77.0


In [17]:
# DEMO - Méthode personnalisée de validation des arguments passés
class ValidationUtils:
    
    def __init__(self, email):
        if self.est_email_valide(email):
            self.email = email
        else:
            raise Exception(f"Input : {email} - Please enter a valid email adress. It should contain an @")
    
    @staticmethod
    def est_email_valide(email):
        return "@" in email

# Utilisation
email = "exemple@domaine.com"
print(ValidationUtils.est_email_valide(email))  # Affiche True


test_1 = ValidationUtils("myemail@email.com")
print("test_1", test_1.email)

test_2 = ValidationUtils("myemail.com")
print("test_2", test_2.email)


True
test_1 myemail@email.com


Exception: Input : myemail.com - Please enter a valid email adress. It should contain an @

In [19]:
# DEMO : Validateur de classe avec Pydantic + staticmethod

from pydantic import BaseModel, Field, field_validator

class Utilisateur(BaseModel):
    nom: str
    age: int

    @field_validator("age")
    @staticmethod
    def verifier_age(age: int) -> int:
        if age < 0:
            raise ValueError("L'âge doit être positif")
        return age

# Utilisation
try:
    utilisateur = Utilisateur(nom="Alice", age=-1)
except ValueError as e:
    print(e)  # Affiche: L'âge doit être positif


1 validation error for Utilisateur
age
  Value error, L'âge doit être positif [type=value_error, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


In [18]:
# DEMO - Méthode de fabrique
class Personne:
    def __init__(self, prenom, nom):
        self.prenom = prenom
        self.nom = nom

    @staticmethod
    def depuis_nom_complet(nom_complet):
        prenom, nom = nom_complet.split()
        return Personne(prenom, nom)

# Utilisation
personne = Personne.depuis_nom_complet("Jean Dupont")
print(f"{personne.prenom} {personne.nom}")  # Affiche Jean Dupont



Jean Dupont


## **@property**

**Fonctionnement** :
Cela permets de définir des méthodes qui peuvent être accédées comme des attributs. Utile pour encapsuler l’accès aux attributs privés et utiliser des méthodes de validation.
La property peut avoir un setter (avec @attribut.setter), getter (directement initialisé avec @property)

**Utilité**:
- Encapsulation : contrôle l'accès aux attributs privés avec les méthodes de validation intégrées
- Calculs automatisés en fonction d'autres attributs
- Code plus propre

In [26]:
class Personne:
    def __init__(self, nom, age):
        self._nom = nom
        self._age = age

    # Act as a getter too
    @property
    def nom(self):
        return self._nom

    # Modify dynamically the value with validation method
    @nom.setter
    def nom(self, nouveau_nom):
        if not nouveau_nom:
            raise ValueError("Le nom ne peut pas être vide")
        self._nom = nouveau_nom

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, nouvel_age):
        if nouvel_age < 0:
            raise ValueError("L'âge doit être positif")
        self._age = nouvel_age

# Utilisation
personne = Personne("Alice", 30)
print(personne.nom)  # Affiche: Alice
personne.nom = "Bob"
print(personne.nom)  # Affiche: Bob



Alice
Bob


## **@abstract method**

Utilité : Utilisé dans les classes abstraites pour indiquer que les sous-classes doivent implémenter cette méthode.

- Désigner des design pattern avec des méthodes qui se répètent selon les classes. (Ex. SKLEARN Fit() Predict() Transform())
- Assure de l'uniformicité sur les noms et ecnourage le polymorphisme pour garder une structure commune entre la classe parent et les classes enfants

In [ ]:
from abc import ABC, abstractmethod

class MaClasseAbstraite(ABC):
    @abstractmethod
    def ma_methode(self):
        pass


## **dataclass**

Utilité : Simplifie la création de classes en générant automatiquement des méthodes spéciales comme __init__, __repr__, et __eq__.

In [25]:
from dataclasses import dataclass

@dataclass
class Exemple:
    x: int
    def __post_init__(self):
        self.x += 1
    y: int

# Utilisation
exemple = Exemple(x=5, y=1)
print(exemple.x, exemple.y)  # Affiche 6


6 1
